In [1]:
# Configuration for paths and models
OPENROUTER_API_KEY = "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"
CHROMA_DB_PATH = r"C:\Users\User\analytics_chroma"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"  # Can be changed as needed


In [3]:
#excel loader
import os
import pandas as pd

def load_excel_files(folder_path):
    sheets_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            xls = pd.ExcelFile(file_path)
            for sheet_name in xls.sheet_names:
                df = xls.parse(sheet_name)
                sheets_data.append({
                    "filename": filename,
                    "sheet": sheet_name,
                    "dataframe": df
                })
    return sheets_data


In [4]:
#processor
def chunk_dataframe(df, max_rows=20):
    chunks = []
    total_rows = len(df)
    for start in range(0, total_rows, max_rows):
        end = min(start + max_rows, total_rows)
        chunk_df = df.iloc[start:end]
        chunk_text = chunk_df.to_csv(index=False)
        chunks.append(chunk_text)
    return chunks

def preprocess_sheets(sheets_data):
    all_chunks = []
    for sheet in sheets_data:
        df = sheet['dataframe'].dropna(how='all')
        chunks = chunk_dataframe(df)
        for chunk in chunks:
            all_chunks.append({
                "filename": sheet["filename"],
                "sheet": sheet["sheet"],
                "content": chunk
            })
    return all_chunks


In [5]:
#embedded
from sentence_transformers import SentenceTransformer

class Embedder:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def encode(self, texts):
        return self.model.encode(texts, show_progress_bar=False)


C:\Users\User\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
import chromadb
from chromadb.config import Settings

def init_chroma_db(persist_path):
    return chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory=persist_path))

def store_chunks(client, collection_name, chunks, embedder):
    collection = client.get_or_create_collection(collection_name)
    for idx, chunk in enumerate(chunks):
        embedding = embedder.encode([chunk["content"]])[0]
        metadata = {
            "filename": chunk["filename"],
            "sheet": chunk["sheet"]
        }
        collection.add(
            documents=[chunk["content"]],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[f"doc_{idx}"]
        )


In [8]:
import requests
from config import OPENROUTER_API_KEY

def query_chroma(collection, embedder, query, top_k=5):
    embedding = embedder.encode([query])[0]
    results = collection.query(query_embeddings=[embedding], n_results=top_k)
    return [doc for doc in results['documents'][0]]

def ask_llm_gemma(context_docs, user_query):
    context = "\n\n".join(context_docs)
    prompt = f"""
You are a business analyst. Use only the following data to answer the question.

Data:
{context}

Question:
{user_query}

Answer:
"""
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    body = {
        "model": "google/gemma-1.5-27b-it",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.2
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", json=body, headers=headers)
    return response.json()["choices"][0]["message"]["content"]


ImportError: cannot import name 'OPENROUTER_API_KEY' from 'config' (C:\Users\User\anaconda3\Lib\site-packages\config\__init__.py)

In [ ]:
from config import *
from excel_loader import load_excel_files
from preprocessor import preprocess_sheets
from embedder import Embedder
from vector_store import init_chroma_db, store_chunks
from query_engine import query_chroma, ask_llm_gemma

def build_pipeline(folder_path, collection_name="excel_data"):
    print("🔄 Loading Excel files...")
    sheets_data = load_excel_files(folder_path)

    print("🧹 Preprocessing sheets...")
    chunks = preprocess_sheets(sheets_data)

    print("🔗 Creating embeddings...")
    embedder = Embedder(EMBEDDING_MODEL_NAME)

    print("💾 Storing to ChromaDB...")
    client = init_chroma_db(CHROMA_DB_PATH)
    store_chunks(client, collection_name, chunks, embedder)

    return client, embedder

def chat_loop(client, embedder, collection_name="excel_data"):
    collection = client.get_collection(collection_name)
    while True:
        user_query = input("\nAsk a question (or type 'exit'): ").strip()
        if user_query.lower() == "exit":
            break
        docs = query_chroma(collection, embedder, user_query)
        answer = ask_llm_gemma(docs, user_query)
        print("\n📊 Answer:")
        print(answer)

if __name__ == "__main__":
    import sys
    if len(sys.argv) < 2:
        print("Usage: python main.py <path_to_excel_folder>")
        exit(1)
    folder = sys.argv[1]
    client, embedder = build_pipeline(folder)
    chat_loop(client, embedder)


In [ ]:
import os
import pandas as pd
import requests
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

# =========[ Configuration ]=========
OPENROUTER_API_KEY = "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"  # <-- PUT YOUR OpenRouter KEY HERE
CHROMA_DB_PATH = r"C:\Users\User\analytics_chroma"
EXCEL_FOLDER_PATH = "./excel_data"  # <-- PUT YOUR .xlsx FILES IN THIS FOLDER
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"

# =========[ Load Excel Files ]=========
def load_excel_files(folder_path):
    sheets_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            xls = pd.ExcelFile(file_path)
            for sheet_name in xls.sheet_names:
                df = xls.parse(sheet_name)
                sheets_data.append({
                    "filename": filename,
                    "sheet": sheet_name,
                    "dataframe": df
                })
    return sheets_data

# =========[ Preprocessing ]=========
def chunk_dataframe(df, max_rows=20):
    chunks = []
    total_rows = len(df)
    for start in range(0, total_rows, max_rows):
        end = min(start + max_rows, total_rows)
        chunk_df = df.iloc[start:end]
        chunk_text = chunk_df.to_csv(index=False)
        chunks.append(chunk_text)
    return chunks

def preprocess_sheets(sheets_data):
    all_chunks = []
    for sheet in sheets_data:
        df = sheet['dataframe'].dropna(how='all')
        chunks = chunk_dataframe(df)
        for chunk in chunks:
            all_chunks.append({
                "filename": sheet["filename"],
                "sheet": sheet["sheet"],
                "content": chunk
            })
    return all_chunks

# =========[ Embedding Model ]=========
class Embedder:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def encode(self, texts):
        return self.model.encode(texts, show_progress_bar=False)

# =========[ ChromaDB Setup ]=========
def init_chroma_db(persist_path):
    return chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory=persist_path))

def store_chunks(client, collection_name, chunks, embedder):
    collection = client.get_or_create_collection(collection_name)
    for idx, chunk in enumerate(chunks):
        embedding = embedder.encode([chunk["content"]])[0]
        metadata = {
            "filename": chunk["filename"],
            "sheet": chunk["sheet"]
        }
        collection.add(
            documents=[chunk["content"]],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[f"doc_{idx}"]
        )

# =========[ Query & LLM Call ]=========
def query_chroma(collection, embedder, query, top_k=5):
    embedding = embedder.encode([query])[0]
    results = collection.query(query_embeddings=[embedding], n_results=top_k)
    return [doc for doc in results['documents'][0]]

def ask_llm_gemma(context_docs, user_query):
    context = "\n\n".join(context_docs)
    prompt = f"""
You are a business analyst. Use only the following data to answer the question.

Data:
{context}

Question:
{user_query}

Answer:
"""
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    body = {
        "model": "google/gemma-1.5-27b-it",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.2
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", json=body, headers=headers)
    return response.json()["choices"][0]["message"]["content"]

# =========[ Main Flow ]=========
def build_pipeline():
    print("📁 Loading Excel files from:", EXCEL_FOLDER_PATH)
    sheets_data = load_excel_files(EXCEL_FOLDER_PATH)

    print("🔧 Preprocessing sheets...")
    chunks = preprocess_sheets(sheets_data)

    print("🧠 Embedding and storing in ChromaDB...")
    embedder = Embedder(EMBEDDING_MODEL_NAME)
    client = init_chroma_db(CHROMA_DB_PATH)
    store_chunks(client, "excel_data", chunks, embedder)

    return client, embedder

def chat_loop(client, embedder):
    collection = client.get_collection("excel_data")
    while True:
        user_query = input("\n❓ Ask a question (or type 'exit'): ").strip()
        if user_query.lower() == "exit":
            break
        docs = query_chroma(collection, embedder, user_query)
        if not docs:
            print("⚠️ No relevant data found.")
            continue
        answer = ask_llm_gemma(docs, user_query)
        print("\n📊 Answer:")
        print(answer)

# =========[ Run ]=========
if __name__ == "__main__":
    if not os.path.exists(EXCEL_FOLDER_PATH):
        os.makedirs(EXCEL_FOLDER_PATH)
        print(f"📂 Created folder '{EXCEL_FOLDER_PATH}'. Please add your .xlsx files there and rerun.")
    else:
        client, embedder = build_pipeline()
        chat_loop(client, embedder)
